# Hidden Markov Models

In this notebook you will find implementations of the four types of inference that we discussed in class. As usual, we first import some important things.

In [ ]:
import numpy as np
from numpy.linalg import matrix_power

Let us define some matrices. Say we have $k$ possible states: $S=\{s_1,s_2\dots,s_k\}$. Then, the **transition matrix** is a $k\times k$ matrix in which the $T_{ij} = P(X_k=s_j|X_{k-1}=s_i)$. For the umbrella example we would have the following:

$$
T=
\begin{bmatrix}
0.7 & 0.3\\
0.3 & 0.7
\end{bmatrix}
$$

As for the $E_k$, let us assume that we have $m$ possible observations: $E=\{e_1,e_2,\dots,e_m\}$. Then, we can define $m$ diagonal matrices such that the ith diagonal entry of matrix $j$, $j=1,2,\dots,m$, is given by $O_{j,ii}=P(E_k=e_j|X_k=s_i)$. For the example we are considering, we would have two matrices:

$$
O_U=
\begin{bmatrix}
0.9 & 0\\
0 & 0.2
\end{bmatrix}, \text{ and }
O_{NU}=
\begin{bmatrix}
0.1 & 0\\
0 & 0.8
\end{bmatrix}.
$$

Notice that, in this case, we have $e_1=\text{Umbrella}$, $e_2=\text{No umbrella}$, $s_1=\text{Rain}$, and $s_2=\text{No rain}$.

Also, we need to define the initial state model: $P(X_0)=[0.5, 0.5]$.

In [ ]:
T = np.array([[0.7, 0.3], [0.3, 0.7]])
O_umbrella = np.array([[0.9, 0], [0, 0.2]])
O_no_umbrella = np.array([[0.1, 0], [0, 0.8]])
X0 = np.array([0.5, 0.5]).reshape((T.shape[0],1))

For convenience, we store the "observation matrices" in a dictionary. Also, we define a dictionary that will contain all the possible states of the system. 

In [ ]:
O = {True: O_umbrella, False: O_no_umbrella}
S = {0: 'Rain', 1: 'No rain'}

As it was mentioned in class, **filtering** is the act of, given some evidence up to a time $t_k$, we want to infer what is the probability distribution of the state $X_k$: $P(X_k|E_{1:k}=e_{1:k})$. The main iteration of this algorithm is the following:

$$
f_{1:k+1}~=\alpha O[k+1]T^Tf_{1:k}~,
$$

where $O[k]$ is the observation matrix for time $t_k$, $f_{1:k}=P(X_k|E_{1:k}=e_{1:k})$, and $\alpha$ is a normalization constant.

In [ ]:
def filtering(E, O, T, X0):
    
    forward = X0
    
    for evidence in E:
        forward = O[evidence] @ T.T @ forward
        forward = forward / forward.sum()
        
    return forward

In [ ]:
E = [True, True]

filtering(E, O, T, X0)

Then, $P(R_2|U_{1:2}=[\text{Umbrella}, \text{Umbrella}])=[0.8833, 0.1166]$.

Regarding **prediction**, we want to compute $P(X_{k+j}|E_{1:k}=e_{1:k})$, $j>0$. This is basically filtering followed by a multiplication of transition matrices.

In [ ]:
def prediction(E, O, T, X0, j):
    
    prediction = np.linalg.matrix_power(T.T, j) @ filtering(E, O, T, X0)
    
    return prediction        

In [ ]:
E = [True, True]

prediction(E, O, T, X0, 12)

It can be seen that $P(X_{2+11}|U_{1:2}=[\text{Umbrella}, \text{Umbrella}]~~)\approx P(X_0)=[0.5,0.5]$.

As for **smoothing**, we wish to obtain $P(X_j|E_{1:k}=e_{1:k})$, $1\leq j<k$. In order to do this, we can do filtering up to the time $t_j$, and combine this with a *backward iteration*, which is given by

$$
b_{m+1:k}=TO[m+1]b_{m+2:k},
$$

where $O[k]$ is the observation matrix for time $t_{j+1}$, and $b_{m+1:k}=P(E_{m+1:k}=e_{m+1:k}|X_k)$.

In [ ]:
def smoothing(E, O, T, X0, k):
    
    forward = filtering(E[:k], O, T, X0)
    
    backward = np.ones((T.shape[0],1))
    
    for evidence in E[-1:k-1:-1]:
        backward = T @ O[evidence] @ backward
    
    smooth = forward * backward
    smooth = smooth / smooth.sum()
    
    return smooth

In [ ]:
E = [True, True]

smoothing(E, O, T, X0, 1)

Therefore, $P(R_1|U_{1:2}=[\text{Umbrella}, \text{Umbrella}])=[0.8833,0.1166]$.

Finally, for obtaining the most likely sequence that produced the given evidence, we need to compute

$$
\max_{x_{1:k}}P(x_{1:k}|E_{1:k}=e_{1:k}).
$$

One option would be to calculate the 32 probabilities that we have since there are 32 possible states when the evidence is a vector of five observations, the most likely sequence is the one that has the highest probability. As you can imagine, this can have very easily a high computational cost. Fortunately, there is an efficient procedure known as the **Viterbi algorithm** with linear complexity $O(k)$. An implementation of the pseudocode of this algorithm, which can be found on https://en.wikipedia.org/wiki/Viterbi_algorithm, is given in the next cell. 

In [ ]:
def most_likely_sequence(E, O, S, T, X0):
    
    sequences = np.zeros((T.shape[0], len(E)))
    states = np.zeros((T.shape[0], len(E)))
    ones = np.ones((T.shape[0], T.shape[0]))
    sequences[:, 0] = (O[E[0]] @ X0).reshape((T.shape[0],))
    message = sequences[:, 0].reshape((T.shape[0], 1))
    
    for i, evidence in enumerate(E[1:]):
        message = (T @ O[evidence]) * (message * ones)
        states[:, i+1] = np.argmax(message, axis=0).reshape((T.shape[0],))
        message = np.max(message, axis=0).reshape((T.shape[0], 1))
        sequences[:, i+1] = message.reshape((T.shape[0],))
        
    states = states.astype('int32')
    s = np.argmax(sequences[:, -1], axis=0)
    best_sequence = [S[s]]
    
    for i in range(len(E)-1, 0, -1):
        s = states[s, i]
        best_sequence.append(S[s])
        
    best_sequence = best_sequence[::-1] 
    
    return best_sequence

In [ ]:
E = [True, True, False, True, False]

most_likely_sequence(E, O, S, T, X0)

# Localization

Hidden Markov Models (HMM) are used in a wide variety of applications such as speech recognition, part-of-speech tagging, machine traslation, handwriting recognition and many more. In this section we will see how HMM can be employed to infer the position of a robot in a static enviroment.

Let us represent the enviroment in which the robot roams around by a grid. Each square of the grid is either free or occupied, in which case we are talking about a position that the robot can not visit. Let $X_k$ be the position of the robot at time $t_k$. Given the latter, all the possible states of $X_k$ are the free spots of the grid: $S\{s_1,s_2,\dots,s_n\}$, where $n$ is the number of free squares. Speaking of "free squares," suppose we do not know where the robot is, so $P(X_0)=1/n$, $\forall s_i\in S$. Also, let $\text{neighnors}(s)$ be the set of empty squares adjacent to $s$ and let $N(s)$ be the size of this set. Then, the transition model of a robot that moves to any adjacent and empty square with equal probability is given by

$$
P(X_{k+1}=s_j|X_k=s_i)=T_{ij}=
\begin{cases}
\frac{1}{N(s_i)}~, & s_j\in \text{neighbors}(s_i)\\
0, & s_j\notin \text{neighbors}(s_i)
\end{cases}
$$

As it was mentioned, the robot will try to know where it is located given a set of observations. In this case, the robot is equipped with four sensors that indicate if there is an obstacle or not in a particular direction. We will assume all the possible directions are North (N), East (E), West (W) and South (S). Further, there is a sensor error rate $\epsilon$ and errors occur independently in the four sensor directions. For instance, the probability of getting all four measurements right is $(1-\epsilon)^4$, the probability of getting them all wrong is $\epsilon^4$. Given this, we have that there are 16 possible measurements for $E_k$ at a given time $t_k$. Also, let $d_{ik}$ be the discrepancy: the number of bits that are different from the true bits for the square $s_i$. Then, the probability that a robot in square $s_i$ would get a reading $e_k$ is equal to

$$
P(E_k=e_k|X_k=s_i)=O_{e_k,ii}=(1-\epsilon)^{4-d_{~ik}}~\epsilon^{~d_{~ik}}.
$$

For instance, the probability that a square with obstacles to the North and South would produce a reading (N,S,E) is equal to $(1-\epsilon)^3~\epsilon$.

Now that we know how to define the transition and sensor models we can carry out different types of inference: we can estimate a location given a set of measurements doing filtering; we can use smoothing to infer a past location given some observations; or we can use the Viterbi algorithm and obtain the most likely path that the robot took and that produced a given set of sensor values.

<img src="robot.png" alt="Drawing" style="width: 700px;"/>

Your mission is to carry out these three types of inference. In particular, for both filtering and smoothing, instead of using something like the figures shown above, use heatmaps in which the color changes depending on the probability of finding the robot at a certain square given a set of measurements. For the most likely path highlight the squares associated to said path. Keep in mind that you will have to simulate the random behavior of the sensors. 